In [16]:
import string, sys
import pandas as pd
import networkx as nx
from scipy.optimize import root, fsolve
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
#%matplotlib inline
#%config InlineBackend.figure_format = 'retina'

In [40]:
# parameters
p_ct = 0.0 # proba that a contact over threshold_ct is actually kept (efficiency of manual CT)
threshold_ct = 900
ndays_delay_ct = 2

threshold_app = 900
ndays_delay_app = 0

p_compliance = 1.  

ntrials = 1000 # number of runs

#threshold_ct = int(sys.argv[1])
#ndays_delay_ct = int(sys.argv[2])
#p_ct = float(sys.argv[3])
#threshold_app = int(sys.argv[4])
#p_compliance = float(sys.argv[5])
# ndays_delay_app = float(sys.argv[6])

delta = 60*15  # timestep
dataset = 'InVS15_1week'

time_exposed = 3.7*86400
time_presympt = 1.5*86400
time_sympt = 2.3*86400

rp = 1    # reduction of beta for presymptomatics
rbeta = 0.5  # reduction of beta for asymptomatics and mildsymptomatics

p_asympt = 0.2
p_mild = 0.72

beta = 1.37e-3

ndays_to_isol = 0.5
time_to_isol = int(ndays_to_isol*86400/delta) # time between start of symptoms and detection and isolation
p_detect = 0.5  # proba to get detected if mild symptoms

delay_ct = int(ndays_delay_ct*86400/delta)   # time before being able to warn using usual CT
delay_app = int(ndays_delay_app*86400/delta)   # time before quarantine with app

ndaystokeep = 2
keeping_duration = int(ndaystokeep*86400./delta)    # number of days in which contact info is kept
duration_isolation = int(14.*86400./delta)

tmax = 100*86400/delta

In [41]:
g = {}
setnodes = set()
totweight = 0.
f = open('tij_' + dataset + '.dat')
(t0,i,j,w) = map(int,f.readline().split())
f.close()

set_w = set()

G_agg = nx.Graph()

f = open('tij_' + dataset + '.dat')
for line in f:
    (t,i,j,w) = map(int,line.split())
    if t not in g:
        g[t] = nx.Graph()
    g[t].add_edge(i,j,weight = w)
    set_w.add(w)
    totweight += w
    setnodes.add(i)
    setnodes.add(j)
    wa = 0
    if G_agg.has_edge(i,j):
        wa = G_agg.get_edge_data(i,j)['weight']
    G_agg.add_edge(i,j,weight= wa+w)
f.close()

t_last = t
n_nodes = len(setnodes)
duration = (t_last -t0)*delta
ndays = int(duration/86400) + 1
length_data_to_loop = int(ndays*86400/delta)
for t in range(length_data_to_loop):
    if t not in g:
        g[t] = nx.Graph()
    for i in setnodes:
        if i not in g[t]:
            g[t].add_node(i)


In [42]:
# creating instantaneous networks before t=0
for t in range(-length_data_to_loop,0):
    g[t] = g[t+length_data_to_loop].copy()

In [43]:
# creating the networks used for the app, which are the aggregated [t-keeping_duration,t]
# need to do it looping the dataset

G_app = {}
for delta_q in [0,0.5,1,1.5,2]:
    keep_dur = keeping_duration - int(delta_q*86400./delta)


    G_app[delta_q] = {}
    G_app[delta_q][-1] = nx.Graph()
    for i in setnodes:
        G_app[delta_q][-1].add_node(i)

    for t in range(length_data_to_loop):
        G_app[delta_q][t] = nx.Graph()
        for i in setnodes:
            G_app[delta_q][t].add_node(i)

        for tback in range(t - keep_dur,t):
            tdata = tback % length_data_to_loop
            for e in g[tdata].edges():            # add events happening at time tback
                w = g[tdata].get_edge_data(e[0],e[1])['weight']
                w_app = 0
                if G_app[delta_q][t].has_edge(e[0],e[1]):
                    w_app = G_app[delta_q][t].get_edge_data(e[0],e[1])['aggduration']

                G_app[delta_q][t].add_edge(e[0],e[1], aggduration = w_app+w)


In [44]:
transmissionprobas = {}
for w in set_w:
    for r in [1,rbeta,rp]:
        transmissionprobas[(w,r)] = 1.-(1-beta*r)**w

In [45]:
def contagionprocess(g_inst, ct, t, status, setS, setE, setIp, setIa, setIm, setIs, 
                     setRam, setRs, setRi, setW,
                     setQuarantined , setIsol,    # quarantined at time t
                     setRQ, setRQS,       # overall set of quarantined individuals, of quarantined while S
                     transmissionprobas, nQ, nQFP):

    new_status = {}
    exposed_to_add, ip_to_add, ia_to_add, im_to_add, is_to_add = set(), set(), set(), set(), set()
    ram_to_add, rs_to_add, ri_to_add, rq_to_add = set(), set(), set(), set()

    s_to_add , r_from_isol_iam_to_add, r_from_isol_is_to_add = set(), set(), set()
    set_warned = set()
    set_to_quarantine = set()
    setIsol_to_add, setIsol_released = set(), set()
   
    # transmission
    for i in setIp.union(setIa).union(setIm).union(setIs):
        r = 1
        if i in setIp:
            if status[i][0] == 'Ip_s':
                r = rp
            else:
                r = rbeta 
        elif i in setIa or i in setIm:
            r = rbeta
        
        for j in g_inst.neighbors(i):
            if status[j] != 'S':
                continue
            w = g_inst.get_edge_data(i,j)['weight']
            if random.uniform(0,1) < transmissionprobas[(w,r)]:
                duration = int(random.normal(time_exposed,time_exposed/10.)/delta)   # duration in exposed phase
                new_status[j] = ('E', t + duration)
                # extract time in which the node will change state again
                exposed_to_add.add(j)
                
    # Evolution from E to Ip
    for i in setE:
      #  print(i,status[i])
        if t > status[i][1]:
            duration = int(random.normal(time_presympt,time_presympt/10.)/delta)    # time of asymptomaticity
            c = random.uniform(0,1)
            if c < p_asympt:
                new_status[i] = ('Ip_a', t+duration)    # presympt becoming asymp
            elif c < p_asympt + p_mild:
                new_status[i] = ('Ip_m', t+duration)     # presympt becoming mildsympt
            else:
                new_status[i] = ('Ip_s', t+duration)    # presympt becoming severesympt            
            ip_to_add.add(i)
    
    # Evolution from Ip to Ia, Is or Im
    for i in setIp:
        if t > status[i][1]:
            duration = int(random.normal(time_sympt,time_sympt/10.)/delta)  # duration of symptoms
            if status[i][0] == 'Ip_a':
                new_status[i] = ('Ia', t+duration)
                ia_to_add.add(i)
            elif status[i][0] == 'Ip_m':
                if 'detected_if_mild' in flags[i]:
                    duration = min(duration,time_to_isol)        # mild cases that are detected will be isolated 
                new_status[i] = ('Im', t+duration)
                im_to_add.add(i)
            else:
                new_status[i] = ('Is', t+min(duration, time_to_isol))   # severe cases will be isolated=>R
                is_to_add.add(i)                   
    
    # Evolution from I (Ia, Im or Is) to R
    for i in setIa:
        if t > status[i][1]:
            new_status[i] = 'R'
            ram_to_add.add(i)
            
    for i in setIm:
        if t > status[i][1]:
            if 'detected_if_mild' in flags[i]:
                new_status[i] = ('Isol_m',t,t+duration_isolation)                
                setIsol_to_add.add(i)    
                # normal CT: warn neighbors above threshold_ct with proba p_ct
                for j in G_app[0][ct].neighbors(i):
                        risk = G_app[0][ct].get_edge_data(i,j)['aggduration']
                        if risk > threshold_ct  and random.uniform(0,1) < p_ct:
                            set_warned.add(j) 
                            t_warning[j] = t + delay_ct
            else:                        # not detected 
                new_status[i] = 'R'
                ram_to_add.add(i)
                
    for i in setIs:
        if t > status[i][1]:                 # detected and isolated after time_to_isol
            new_status[i] = ('Isol_s',t,t+duration_isolation)
            setIsol_to_add.add(i)
            # normal CT: warn neighbors above threshold_ct with proba p_ct
            for j in G_app[0][ct].neighbors(i):
                risk = G_app[0][ct].get_edge_data(i,j)['aggduration']
                if risk > threshold_ct and random.uniform(0,1) < p_ct:
                        set_warned.add(j) 
                        t_warning[j] = t +delay_ct
                     
    for i in setIsol:
        if t > status[i][2]:   # liberated after duration_isolation
            new_status[i] = ('RI',t-duration_isolation)
            ri_to_add.add(i)
            setIsol_released.add(i)
            if status[i][0] == 'Isol_m':
                ram_to_add.add(i)
            else:
                rs_to_add.add(i)
                
# Evolution within quarantine and exit from quarantine
    for i in setQuarantined:          
        if t > status[i][1]:          
            if status[i][0] == 'S_Q':    # end of isolation
                new_status[i] = 'S'
                s_to_add.add(i)
            elif status[i][0] == 'R_Q':
                new_status[i] = 'R'
                r_from_isol_iam_to_add.add(i)
            elif status[i][0] == 'RI_Q':
                ti = status[i][2]
                new_status[i] = ('RI',ti)
                ri_to_add.add(i)
            elif status[i][0] == 'E_Q':   # transition to Ip_Q
                duration = int(random.normal(time_presympt,time_presympt/10.)/delta)    # time of asymptomaticity
                c = random.uniform(0,1)
                if c < p_asympt:
                    new_status[i] = ('Ip_a_Q', t+duration, status[i][2])    # presympt becoming asymp
                elif c < p_asympt + p_mild:
                    new_status[i] = ('Ip_m_Q', t+duration, status[i][2])     # presympt becoming mildsympt
                else:
                    new_status[i] = ('Ip_s_Q', t+duration, status[i][2])    # presympt becoming severesympt   
            elif status[i][0] ==  'Ip_a_Q':                    # transition to Ix_Q
                duration = int(random.normal(time_sympt,time_sympt/10.)/delta)
                new_status[i] = ('Ia_Q', t+duration)
            elif status[i][0] == 'Ip_m_Q':
                duration = int(random.normal(time_sympt,time_sympt/10.)/delta)
                new_status[i] = ('Im_Q', t+min(duration, time_to_isol), status[i][2])   # mild, quarantined cases will be detected
            elif status[i][0] == 'Ip_s_Q':
                duration = int(random.normal(time_sympt,time_sympt/10.)/delta)
                new_status[i] = ('Is_Q', t+min(duration, time_to_isol), status[i][2])   # severe cases will be detected
            elif status[i][0] ==  'Ia_Q':
                new_status[i] = 'R'
                r_from_isol_iam_to_add.add(i)
                rq_to_add.add(i)
            elif status[i][0] == 'Im_Q' or status[i][0] == 'Is_Q':   # detected hence warn contacts before quarantine
                rq_to_add.add(i)
                if status[i][0] == 'Im_Q':
                    r_from_isol_iam_to_add.add(i)
                else:
                    r_from_isol_is_to_add.add(i)
                tq = status[i][2]                # time at which i was quarantined
                new_status[i] = ('R',tq)
                dt = (t-tq) % length_data_to_loop
                if dt >= keeping_duration:     # then no contacts to warn
                    continue 
                b = dt // (86400/2/delta)   *0.5        # t-tq in units of half-days
                ctq = tq % length_data_to_loop
                # normal CT: warn neighbors above threshold_ct
                for j in G_app[b][ctq].neighbors(i):
                    risk = G_app[b][ctq].get_edge_data(i,j)['aggduration']
                    if risk > threshold_ct  and random.uniform(0,1) < p_ct:
                        set_warned.add(j) 
                        t_warning[j] = t + delay_ct
    
    
    setW_to_remove = set()
    for i in setW:
        if t > t_warning[i]:
            set_to_quarantine.add(i)
            setW_to_remove.add(i)
            
    setW = setW - setW_to_remove
    
    #update of status and set of infectious
    for i in new_status:
        status[i] = new_status[i]    
        
    setS = setS.union(s_to_add) - exposed_to_add   
    setE = setE.union(exposed_to_add) - ip_to_add   
    
    setIp = setIp.union(ip_to_add) - ia_to_add - im_to_add - is_to_add   
    
    setIa = setIa.union(ia_to_add) - ram_to_add
    setIm  = setIm.union(im_to_add) - ram_to_add  - setIsol_to_add
    setIs = setIs.union(is_to_add) - rs_to_add - setIsol_to_add
    
    setRam = setRam.union(ram_to_add).union(r_from_isol_iam_to_add)
    setRs = setRs.union(rs_to_add).union(r_from_isol_is_to_add)    
    setRi = setRi.union(ri_to_add)
    setRQ = setRQ.union(rq_to_add)
    
    setIsol = setIsol.union(setIsol_to_add) - setIsol_released    
    setQuarantined = setQuarantined - s_to_add - r_from_isol_iam_to_add - r_from_isol_is_to_add
 

# send warning from recently isolated people with app (RI such that t_isol > t-keeping_duration) to neighbors with app
# then compute risk for these
    riskscore = {}
    for i in setRi.union(setRQ).union(setIsol):
        if 'app' not in flags[i]:
            continue
        if len(status[i]) < 2:
            continue
        ti = status[i][1]
        if ti <= t - keeping_duration:    # i was quarantined or isolated long ago
            continue
        b = (t-ti) // (86400/2/delta)   *0.5        # t-ti in units of half-days
        ctq = ti % length_data_to_loop
        for j in G_app[b][ctq].neighbors(i):      
            if 'app' not in flags[j]:   # j does not have app
                continue
            if j not in riskscore:
                riskscore[j] = 0
            riskscore[j] += G_app[b][ctq].get_edge_data(i,j)['aggduration'] 
            
    for j in riskscore:
        if riskscore[j] > threshold_app:
            set_warned.add(j)
            t_warning[j] = t + delay_app


########### now treating the warned 
    # what happens with warned individuals
    for i in set_warned:  # warned by the app  
        if i in setQuarantined:
            continue
        if i in setIsol:
            continue
        if 'warned' not in flags[i]:
            flags[i].append('warned')
        if 'compliant' in flags[i]:     # quarantines for duration_isolation, starting immediately
            setW.add(i)
        else:       # not compliant: if S or Ia, nothing happens except becoming "detected_if_mild"; 
                    #   if mild or severe symptoms, report after time_to_isol/2  
                    #   if E or presymptomatics, add 'detected if mild'
            if i in setS.union(setIa).union(setE).union(setIp):
                if 'detected_if_mild' not in flags[i]:
                    flags[i].append('detected_if_mild')
            if i in setIm.union(setIs):
              #  print(status[i])
                status[i] = (status[i][0],min(status[i][1], t + time_to_isol/2) )
                if 'detected_if_mild' not in flags[i]:
                    flags[i].append('detected_if_mild')
    
    for i in set_to_quarantine:
            if i in setQuarantined:
                continue       
            setQuarantined.add(i)    
            nQ += 1
            if i in setRam.union(setRs):   # i already recovered hence no action
                nQFP += 1
                if status[i] == 'R':
                    status[i] = ('R_Q',t+duration_isolation)   
                else:
                    tq = status[i][1]
                    status[i] = ('RI_Q',t+duration_isolation,tq)
                if i in setRi:
                    setRi.remove(i)
                if i in setRQ:
                    setRQ.remove(i)
            if i in setS:
                nQFP += 1
                setRQS.add(i)          # Susceptible being quarantined
                setS.remove(i)
                status[i] = ('S_Q',t+duration_isolation,t)
                if 'detected_if_mild' not in flags[i]:
                    flags[i].append('detected_if_mild')
            else:
              #  print(status[i])
                tnext = status[i][1]
                if i in setE:
                    setE.remove(i)
                    status[i] = ('E_Q',tnext,t)
                elif i in setIp:
                    setIp.remove(i)
                    status[i] = (status[i][0] + '_Q',tnext,t)
                elif i in setIa:
                    setIa.remove(i)
                    status[i] = ('Ia_Q',tnext,t)
                elif i in setIm:
                    setIm.remove(i)
                    status[i] = ('Im_Q',tnext,t)
                elif i in setIs:
                    setIs.remove(i)
                    status[i] = ('Is_Q',tnext,t)
                
    set_warned = set()

    return setS, setE, setIp, setIa, setIm, setIs, setRam, setRs, setRi, setW, setQuarantined, setIsol, setRQ, setRQS, nQ, nQFP



In [46]:
smalloutbreak = 0.1 # separation small/large outbreak

wfrac = open('results_isol_ct_app_delayct%s_delayapp%s_pct%s_threshold%s_%s_compliance_%s.dat' % 
                        (ndays_delay_ct,ndays_delay_app, p_ct, threshold_ct,threshold_app,p_compliance) ,'w')

wpct= str(round(p_ct,2))

for f_adopt in np.linspace(1,0.,11):

    wf = str(round(f_adopt,2))
    
    list_S, list_E, list_Ip, list_Iam, list_Is, list_Ram, list_Rs, list_Ri  = {}, {}, {}, {}, {}, {}, {}, {}
    list_Q, list_RQ, list_RQS = {}, {}, {}
    nQ_ave, nQFP_ave, ratioFP_ave, norm_rFP = 0, 0,0,0
    
    for itrial in range(1,ntrials+1):

        status = {}
        setS, setE, setIp, setIa, setIm, setIs = set(), set(), set(), set(), set(), set()
        setRam, setRs, setRi, setRQ, setRQS  = set(), set(), set(), set(), set()
        setQuarantined, setW , setIsol = set(), set(), set()
        t_warning = {}
        flags = {}
        nQ = 0 # number of quarantines (same individual can count more than once)
        nQFP = 0 # number of S or R individuals sent into quarantine
        
        for i in setnodes:
            status[i] = 'S'
            setS.add(i)
            flags[i] = []
            if random.uniform(0,1) < p_detect:
                flags[i].append('detected_if_mild')
            if random.uniform(0,1) < p_compliance:
                    flags[i].append('compliant')  
                    
        setadopt = set()
        while len(setadopt) < f_adopt*n_nodes:
            i = random.choice(list(setnodes))
            setadopt.add(i)
            flags[i].append('app')


        real_current_time = 0
        current_time = 0
        # initial seed
        i0 = random.choice(list(setnodes))
        # choice of nexttime
        duration = int(random.normal(time_exposed,time_exposed/10.)/delta)
        status[i0] = ('E',current_time + duration)
        setS.remove(i0)
        setE.add(i0)

        while len(setS.union(setRam).union(setRs)) < n_nodes and real_current_time < tmax:
            current_time = real_current_time % length_data_to_loop

            setS, setE, setIp, setIa, setIm, setIs, setRam, setRs, setRi, setW, setQuarantined, setIsol, setRQ, setRQS, nQ, nQFP = \
                                    contagionprocess(g[current_time],  current_time, real_current_time,
                                        status, setS, setE, setIp, setIa,  setIm, setIs, 
                                                     setRam, setRs , setRi, setW,
                                                     setQuarantined, setIsol, setRQ, setRQS, transmissionprobas, nQ, nQFP)

            if real_current_time % 10 == 0:
                if real_current_time not in list_S:
                    list_S[real_current_time] = []
                    list_E[real_current_time] = []
                    list_Ip[real_current_time] = []
                    list_Iam[real_current_time] = []
                    list_Is[real_current_time] = []
                    list_Ram[real_current_time] = []
                    list_Rs[real_current_time] = []
                    list_Ri[real_current_time] = []
                    list_Q[real_current_time] = []
                    list_RQ[real_current_time] = []
                    list_RQS[real_current_time] = []

                    
                list_S[real_current_time].append(len(setS))
                list_E[real_current_time].append(len(setE))
                list_Ip[real_current_time].append(len(setIp))
                list_Iam[real_current_time].append(len(setIa.union(setIm)))
                list_Is[real_current_time].append(len(setIs))
                list_Ram[real_current_time].append(len(setRam))
                list_Rs[real_current_time].append(len(setRs))
                list_Ri[real_current_time].append(len(setRi))
                list_Q[real_current_time].append(len(setQuarantined))
                list_RQ[real_current_time].append(len(setRQ.union(setRQS)))
                list_RQS[real_current_time].append(len(setRQS))
               
            real_current_time += 1

        while real_current_time <= tmax:
            if real_current_time % 10 == 0:
                if real_current_time not in list_S:
                    list_S[real_current_time] = []
                    list_E[real_current_time] = []
                    list_Ip[real_current_time] = []
                    list_Iam[real_current_time] = []
                    list_Is[real_current_time] = []
                    list_Ram[real_current_time] = []
                    list_Rs[real_current_time] = []
                    list_Ri[real_current_time] = []
                    list_Q[real_current_time] = []
                    list_RQ[real_current_time] = []
                    list_RQS[real_current_time] = []

                list_S[real_current_time].append(len(setS))
                list_E[real_current_time].append(len(setE))
                list_Ip[real_current_time].append(len(setIp))
                list_Iam[real_current_time].append(len(setIa.union(setIm)))
                list_Is[real_current_time].append(len(setIs))
                list_Ram[real_current_time].append(len(setRam))
                list_Rs[real_current_time].append(len(setRs))
                list_Ri[real_current_time].append(len(setRi))
                list_Q[real_current_time].append(len(setQuarantined))
                list_RQ[real_current_time].append(len(setRQ.union(setRQS)))
                list_RQS[real_current_time].append(len(setRQS))
    
            real_current_time += 1
 
        nQ_ave += nQ 
        nQFP_ave += nQFP 
        if nQ > 0:
            ratioFP_ave += nQFP/nQ
            norm_rFP += 1    
        
    n_smalloutbreak = 0
        
    for i in range(ntrials):
        r1 = (list_Ram[tmax][i] + list_Iam[tmax][i])/n_nodes 
        r2 = (list_Rs[tmax][i] + list_Is[tmax][i])/n_nodes        
        rtot = r1+r2
        if rtot < smalloutbreak:
            n_smalloutbreak += 1


    
    rtot_ave =  (np.average(list_Ip[tmax]) + np.average(list_Iam[tmax]) +
                        np.average(list_Is[tmax]) + np.average(list_Ram[tmax]) + np.average(list_Rs[tmax]) ) / n_nodes
    rs_ave = (np.average(list_Is[tmax]) + np.average(list_Rs[tmax]) ) / n_nodes
            
    q_ave = np.average(list_RQ[tmax])/n_nodes
    qs_ave = np.average(list_RQS[tmax])/n_nodes

    # write results as a function of p_app: fraction of small outbreaks, average epidemic size, quarantines, 
    # unnecessary quarantines, fraction of unnecesary quarantines
    nQ_ave = nQ_ave/ ntrials 
    nQFP_ave = nQFP_ave/ntrials
    if norm_rFP > 0:
        ratioFP_ave = ratioFP_ave / norm_rFP
        
    wfrac.write(('%s\t'*4+'%s\n') % (wf,n_smalloutbreak/ntrials, rtot_ave, nQ_ave/n_nodes, nQFP_ave/n_nodes))
    
wfrac.close()